In [1]:
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import time

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [9]:
def draw_landmarks(frame, reuslts):
    # draw face landmarks
    mp_drawing.draw_landmarks(frame, reuslts.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=2))
    # draw pose landmarks
    mp_drawing.draw_landmarks(frame, reuslts.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
    # draw left hand landmarks
    mp_drawing.draw_landmarks(frame, reuslts.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    # draw right hand landmarks
    mp_drawing.draw_landmarks(frame, reuslts.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [10]:
cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()

        if not ret:
            print("Camera is not accessible")
            break

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)

        # convert image back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks
        draw_landmarks(image, results)

        cv2.imshow('MediaPipe Holistic', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()

In [16]:
cam.release()
cv2.destroyAllWindows()

: 

In [12]:
def extract_keypoints(results):
    pose  = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3).flatten()
    lh = np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3).flatten()
    rh = np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3).flatten()
    return np.concatenate([pose, face, lh, rh])

In [13]:
# path for the dataset
DATA_PATH = os.path.join('MP_Data')

# actions that we try to detect
actions = np.array(['hello', 'thanks', 'yes', 'no', 'iloveyou'])

# 30 videos worth of data
no_sequences = 30

# 30 frames per video
sequence_length = 30

In [14]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

Collecting dataset

In [15]:
cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                ret, frame = cam.read()

                if not ret:
                    print("Camera is not accessible")
                    break

                # Recolor feed
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(image)

                # convert image back to BGR
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Draw landmarks
                draw_landmarks(image, results)

                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.imshow('MediaPipe Holistic', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.imshow('MediaPipe Holistic', image)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # break if 'q' is pressed
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

cam.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [8]:
cam.release()
cv2.destroyAllWindows()

preprocess data and create albels and features

In [10]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

# load data
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])                        

In [12]:
X = np.array(sequences)

In [13]:
y = tf.keras.utils.to_categorical(labels).astype(int)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

Build and train LSTM Neural network

In [16]:
log_dir = os.path.join('Logs')
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

In [17]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(tf.keras.layers.LSTM(128, return_sequences=True, activation='relu'))
model.add(tf.keras.layers.LSTM(64, return_sequences=False, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(actions.shape[0], activation='softmax'))

c:\Users\vithustennysan\Desktop\Sign Language\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [19]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step - categorical_accuracy: 0.1984 - loss: 2.1407
Epoch 2/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - categorical_accuracy: 0.2051 - loss: 9.8358 
Epoch 3/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - categorical_accuracy: 0.1819 - loss: 1.7981
Epoch 4/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - categorical_accuracy: 0.1691 - loss: 2.0551
Epoch 5/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - categorical_accuracy: 0.1899 - loss: 1.6811
Epoch 6/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - categorical_accuracy: 0.2850 - loss: 1.5931
Epoch 7/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - categorical_accuracy: 0.2763 - loss: 1.6157
Epoch 8/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - categorical_accuracy: 0.2832 - loss: 1.5466
Epoch 9/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - categorical_accuracy: 0.3148 - loss: 1.5056
Epoch 10/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - categorical_accuracy: 0.3122 - loss: 1.4085
Epoch 11/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 64)         │       442,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 30, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,790,225 (6.83 MB)

 Trainable params: 596,741 (2.28 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,193,484 (4.55 MB)

Make predictions

In [21]:
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step


In [23]:
actions[np.argmax(res[0])]

'hello'

In [25]:
actions[np.argmax(y_test[0])]

'hello'

save weights

In [26]:
model.save('action.h5')

Evaluation using Confusion Matrix and Accuracy

In [33]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


In [34]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 4]],

       [[6, 0],
        [0, 2]],

       [[7, 0],
        [0, 1]],

       [[7, 0],
        [0, 1]]], dtype=int64)

In [35]:
accuracy_score(ytrue, yhat)

1.0

Tesing

In [7]:
sequence = []
sentence = []
threshold = 0.5

cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()

        if not ret:
            print("Camera is not accessible")
            break

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)

        # convert image back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks
        draw_landmarks(image, results)

        # Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            sentence.append(actions[np.argmax(res)])  

        if res[np.argmax(res)] > threshold: 
            if len(sentence) > 15:
                if sentence[-1] != actions[np.argmax(res)]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])

        if len(sentence) > 5:
            sentence = sentence[-5:]
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('MediaPipe Holistic', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()

NameError: name 'res' is not defined